In [ ]:
# in this code we take the glofas data (after concatenating, i.e. running glofas_concat.ipynb)
# and prepare the dataset to run the cime mapping files for cesm.
# We are using the JRA55 runoff as the base format.

#JRA example data file: /glade/p/cesmdata/cseg/inputdata/lnd/dlnd7/JRA55/JRA.v1.1.runoff.2016.170807.nc

# Dimensions: bnds: 2latitude: 720longitude: 1440time: 365
# Coordinates:
# time (time) object 2016-01-01 12:00:00 ... 2016-12-...
# longitude (longitude) float64 0.125 0.375 0.625 ... 359.6 359.9
# latitude (latitude) float64 -89.88 -89.62 ... 89.62 89.88
# Data variables:
# rofl (time, latitude, longitude) float32 ...   (kg/m2/sec)
# time_bnds (time, bnds) object ...
# rofi (time, latitude, longitude) float32...

# a few key things to do: make sure units are the same, eliminate upstream area from Glofas.

In [1]:
import xarray as xr
import numpy as np

ds = xr.open_dataset('/glade/work/gseijo/GloFAS/GloFAS_2000_2014.nc')
print(ds)

<xarray.Dataset>
Dimensions:     (latitude: 502, longitude: 1402, time: 5479)
Coordinates:
  * time        (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2014-12-31
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 40.05 39.95 39.85 ... -9.85 -9.95 -10.05
  * longitude   (longitude) float64 -110.1 -110.0 -109.9 ... 29.85 29.95 30.05
    valid_time  (time) datetime64[ns] ...
Data variables:
    dis24       (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-07-12T14:00 GRIB to CDM+CF via cfgrib-0.9.1...


In [2]:
#subset to a region slightly larger than Carib domain for efficiency
glofas = ds.rename({'latitude':'lat','longitude':'lon'})
subset = dict(lat=slice(35, -7), lon=slice(-110, -34))
glofas = glofas.sel(**subset)
glofas['lon'] = glofas['lon'] % 360
glofas= glofas.sortby(glofas.lon)

uparea = xr.open_dataarray('/glade/work/gseijo/python_codes/rivers/upArea.nc').sel(**subset)
uparea['lon'] = uparea['lon'] % 360
uparea = uparea.sortby(uparea.lon)


In [3]:
# change runoff units:
# Convert m3/s to kg/m2/s
# Borrowed from https://xgcm.readthedocs.io/en/latest/xgcm-examples/05_autogenerate.html
distance_1deg_equator = 111000.0
dlon = dlat = 0.1  # GloFAS grid spacing
dx = dlon * xr.ufuncs.cos(xr.ufuncs.deg2rad(glofas.lat)) * distance_1deg_equator
dy = ((glofas.lon * 0) + 1) * dlat * distance_1deg_equator
glofas_area = dx * dy
glofas_rof = glofas.dis24 * 1000.0 / glofas_area
#glofas_rof[0,:,:].plot()

In [4]:
#generate land/ocean mask (runoff = nan in ocean, values in land). We want 0 for land, 1 for ocean.
land_ocean_mask = np.zeros(np.shape(glofas_rof[0,:,:]))
xx, yy = np.shape(land_ocean_mask)
for x in range(xx):
    for y in range(yy):
        if glofas_rof[0,x,y]>=0:
            land_ocean_mask[x,y]=0
        elif np.isnan(glofas_rof[0,x,y]):
            land_ocean_mask[x,y]=1
#plt.pcolormesh(glofas.lon,glofas.lat,land_ocean_mask)

In [ ]:
# Find river end points by looking for local maxima in upstream area (method by J. Simikins)
uparea = uparea.fillna(0).values
points = np.zeros_like(uparea)
window = 2  # look with +- this number of grid points
ni, nj = uparea.shape
for i in range(window, ni-window):
    for j in range(window, nj-window):
        sub = uparea[i-window:i+window+1, j-window:j+window+1]
        point = uparea[i, j]
        # A river end point has a reasonably large upstream area
        # and is a local maximum
        if point > 1e6 and sub.max() == point:
            points[i, j] = 1
glofas_mask = points
glofas_rof = glofas_rof.where(glofas_mask > 0).fillna(0.0)

In [9]:
#create dataset with correct structure and save
ds_new = xr.Dataset({
    'runoff':(['time','latitude','longitude'],glofas_rof),
    'land_ocean_mask':(['latitude','longitude'],land_ocean_mask)},
    coords={'time': glofas['time'].data,'latitude':glofas['lat'].data,'longitude':glofas['lon'].data})

# Drop '_FillValue' from all variables when writing out
all_vars = list(ds_new.data_vars.keys()) + list(ds_new.coords.keys())
encodings = {v: {'_FillValue': 1.0e20} for v in all_vars}

# Make sure time has the right units and datatype
# otherwise it will become an int and MOM will fail. 
attrsT = { 'standard_name': 'time',
             'long_name': 'time',
             'axis': "T",
             'units' : "days since 2000-01-01 00:00:00",
             'calendar' : "gregorian"}

ds_new['time'] = np.linspace(0,len(ds_new.time)-1,len(ds_new.time)) 
ds_new.time.attrs = attrsT
ds_new['latitude'].attrs = {'units': 'degrees_north'}
ds_new['longitude'].attrs = {'units': 'degrees_east'}
ds_new['runoff'].attrs = {'units': 'kg m-2 s-1'}

print(ds_new)
ds_new.to_netcdf('/glade/work/gseijo/GloFAS/GloFAS_2000_2014_cesm.nc',encoding=encodings,unlimited_dims='time')

<xarray.Dataset>
Dimensions:          (latitude: 420, longitude: 760, time: 5479)
Coordinates:
  * time             (time) float64 0.0 1.0 2.0 ... 5.477e+03 5.478e+03
  * latitude         (latitude) float64 34.95 34.85 34.75 ... -6.75 -6.85 -6.95
  * longitude        (longitude) float64 250.0 250.1 250.2 ... 325.7 325.8 325.9
Data variables:
    runoff           (time, latitude, longitude) float64 0.0 0.0 0.0 ... 0.0 0.0
    land_ocean_mask  (latitude, longitude) float64 0.0 0.0 0.0 ... 1.0 1.0 1.0


In [24]:
#create a similar file for mapping purposes (need to create a mask and only need one tstep)
# example file:
#     <xarray.Dataset>
# Dimensions:    (bnds: 2, latitude: 720, longitude: 1440, time: 1)
# Coordinates:
#   * time       (time) object 1988-07-02 12:00:00
#   * longitude  (longitude) float64 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
#   * latitude   (latitude) float64 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
# Dimensions without coordinates: bnds
# Data variables:
#     rofl       (time, latitude, longitude) float32 ...
#     rofi       (time, latitude, longitude) float32 ...
#     time_bnds  (time, bnds) object ...
#     mask       (latitude, longitude) float64 ...
# Attributes:
#     title:                     JRA-55 based river runoff data
#     Source:                    Tsujino et al., 2017. JRA-55 Based Surface Dat...
#     Conventions:               CF1.0
#     history:                   Wed Feb 13 10:56:25 2019: ncra JRA.v1.4.runoff...
#     notes:                     JRA-55 runoff data (v1.4) reformatted for use ...
#     NCO:                       netCDF Operators version 4.7.4 (http://nco.sf....
#     nco_openmp_thread_number:  1

#define region of Caribbean domain:
carib = xr.open_dataset('/glade/work/gseijo/caribbean_cesm/scripts/ocean_hgrid.nc')
minlat = carib.y[0,0]
maxlat = carib.y[-1,0]
minlon = carib.x[0,0] % 360
maxlon = carib.x[0,-1] % 360

mask = np.zeros(np.shape(glofas_rof[0,:,:]))
for i in range (0,len(ds_new.longitude)-1):
    for j in range (0,len(ds_new.latitude)-1):
        if ds_new.longitude[i] >= minlon and ds_new.longitude[i]<= maxlon and ds_new.latitude[j]>= minlat and ds_new.latitude[j]<= maxlat: 
            mask[j,i] = 1
        else:
            mask[j,i] = 0


ds_masked = xr.Dataset({
    'runoff':(['time','latitude','longitude'],(glofas.dis24[0:1,:,:] * 1000.0 / glofas_area).where(glofas_mask > 0).fillna(0.0)),
    'land_ocean_mask':(['latitude','longitude'],land_ocean_mask),
    'mask':(['latitude','longitude'],mask)},
    coords={'time':glofas['time'][0:1].data,'latitude':glofas['lat'].data,'longitude':glofas['lon'].data})

# Drop '_FillValue' from all variables when writing out
all_vars = list(ds_masked.data_vars.keys()) + list(ds_new.coords.keys())
encodings = {v: {'_FillValue': None} for v in all_vars}


encodings['time'].update({
'units': "days since 2000-01-01 00:00:00",
'dtype': float, 
'calendar': 'gregorian'
})
ds_masked['time'] = np.linspace(0,len(ds_masked.time)-1,len(ds_masked.time)) 
ds_masked['time'].attrs = {'cartesian_axis': 'T'}
ds_masked['latitude'].attrs = {'units': 'degrees_north'}
ds_masked['longitude'].attrs = {'units': 'degrees_east'}
ds_masked['runoff'].attrs = {'units': 'kg m-2 s-1'}

print(ds_masked)   
ds_masked.to_netcdf('/glade/work/gseijo/GloFAS/glofas_runoff_masked.nc')

<xarray.Dataset>
Dimensions:          (latitude: 420, longitude: 760, time: 1)
Coordinates:
  * time             (time) float64 0.0
  * latitude         (latitude) float64 34.95 34.85 34.75 ... -6.75 -6.85 -6.95
  * longitude        (longitude) float64 250.0 250.1 250.2 ... 325.7 325.8 325.9
Data variables:
    runoff           (time, latitude, longitude) float64 0.0 0.0 0.0 ... 0.0 0.0
    land_ocean_mask  (latitude, longitude) float64 0.0 0.0 0.0 ... 1.0 1.0 1.0
    mask             (latitude, longitude) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
